# Generate a mean-field (orbital) and CI representation for the oxygen ground configuration

In [16]:
using JAC

Various *ab-initio* many-electron methods have been developed over the years to describe the level structure and properties of atoms and ions with -- more or less -- simple shell structures. These methods include configurations interaction (CI), many-body perturbation theory (MBPT), coupled-cluster (CC) theory, correlation potential (CP), the multiconfigurational Dirac-Hartree Fock (MCDF) representations and are all typically based on some *self-consistent-field* (SCF).  In JAC, we distinguish between different representations of atomic wave (and Green) functions to facilitate the access to the different methods. For the sake of convinience, we here include also the concept of a self-consistent orbital set as perhaps the most basic *representation* of atomic levels and multiplets.

In this tutorial, we shall show how one can easily generate a mean-field (orbital) representation as well as a CI representation, making use of these orbitals. We shall consider the levels of the $1s^2 2s^2 2p^4$ ground configuration of oxygen as a simple but typical example and shall later extent this concept to perform a CI expansion with (virtual) excitations into the $n=3$ shells.

The computations of all *atomic representations* are controlled by the `struct AtomicState.Representation` which is based on a name, nuclear model, (radial) grid as well as set of *reference configurations*. Let us first consider how such a representation is defined inernally:

In [2]:
? AtomicState.Representation

`struct  AtomicState.Representation`       ... a struct for defining an atomic state representation. Such representations often refer to approximate wave function approximations of         one or several levels but may concern also a mean-field basis (for some multiplet of some given configurations) or Green functions,         etc.

```
+ name             ::String                      ... to assign a name to the given model.
+ nuclearModel     ::Nuclear.Model               ... Model, charge and parameters of the nucleus.
+ grid             ::Radial.Grid                 ... The radial grid to be used for the computation.
+ refConfigs       ::Array{Configuration,1}      ... List of references configurations, at least 1.
+ repType          ::AbstractRepresentationType  ... Specifies the particular representation.
```

---

`AtomicState.Representation()`  ... constructor for an 'empty' instance of the a variable::AtomicState.Representation

---

`AtomicState.Representation( ... example for the generation of a mean-field basis)`  

```
    name        = "Oxygen 1s^2 2s^2 2p^4 ground configuration"
    grid        = Radial.Grid(true)
    nuclearM    = Nuclear.Model(8.)
    refConfigs  = [Configuration("[He] 2s^2 2p^4")]
    mfSettings  = MeanFieldSettings()
    Representation(name, nuclearM, grid, refConfigs, MeanFieldBasis(mfSettings) )
```

`AtomicState.Representation( ... example for the computation of a configuration-interaction (CI) expansion)`  

```
    name        = "Oxygen 1s^2 2s^2 2p^4 ground configuration"
    grid        = Radial.Grid(true)
    nuclearM    = Nuclear.Model(8.)
    refConfigs  = [Configuration("[He] 2s^2 2p^4")]
    orbitals    = wb["mean-field basis"].orbitals #   get a proper set of orbitals
    ciSettings  = CiSettings(true, false, Int64[], false, LevelSymmetry[] )
    from        = [Shell("2s")]
    to          = [Shell("2s"), Shell("2p")]
    excitations = RasStep(RasStep(), seFrom=from, seTo=to, deFrom=from, deTo=to, frozen=[Shell("1s")])
    Representation(name, nuclearM, grid, refConfigs, CiExpansion(orbitals, excitations, ciSettings) )
```

`AtomicState.Representation( ... example for the computation of a restricted-active-space (RAS) expansion)`  

```
    name        = "Beryllium 1s^2 2s^2 ^1S_0 ground state"
    refConfigs  = [Configuration("[He] 2s^2")]
    rasSettings = RasSettings([1], 24, 1.0e-6, CoulombInteraction(), true, [1,2,3] )
    from        = [Shell("2s")]
    
    frozen      = [Shell("1s")]
    to          = [Shell("2s"), Shell("2p")]
    step1       = RasStep(RasStep(), seFrom=from, seTo=deepcopy(to), deFrom=from, deTo=deepcopy(to), 
                          frozen=deepcopy(frozen))

    append!(frozen, [Shell("2s"), Shell("2p")])
    append!(to,     [Shell("3s"), Shell("3p"), Shell("3d")])
    step2       = RasStep(step1; seTo=deepcopy(to), deTo=deepcopy(to), frozen=deepcopy(frozen))

    append!(frozen, [Shell("3s"), Shell("3p"), Shell("3d")])
    append!(to,     [Shell("4s"), Shell("4p"), Shell("4d"), Shell("4f")])
    step3       = RasStep(step2, seTo=deepcopy(to), deTo=deepcopy(to), frozen=deepcopy(frozen))

    Representation(name, Nuclear.Model(4.), Radial.Grid(true), refConfigs, 
                   RasExpansion(LevelSymmetry(0, Basics.plus), 4, [step1, step2, step3], rasSettings) )
```

`AtomicState.Representation( ... example for the computation of Green(function) expansion)`  

```
    name            = "Lithium 1s^2 2s ground configuration"
    refConfigs      = [Configuration("[He] 2s")]
    levelSymmetries = [LevelSymmetry(1//2, Basics.plus), LevelSymmetry(3//2, Basics.plus)]
    greenSettings   = GreenSettings(5, [0, 1, 2], 0.01, true, false, Int64[])
    Representation(name, Nuclear.Model(8.), Radial.Grid(true), refConfigs, 
                   GreenExpansion( AtomicState.DampedSpaceCI(), Basics.DeExciteSingleElectron(), 
                   levelSymmetries, 3, greenSettings) ) 
                   
... These simple examples can be further improved by overwriting the corresponding parameters.
```


The particular choice of an atomic representation is made by the `repType::AbstractRepresentationType` which presently may take the following parameters:

In [3]:
? AtomicState.AbstractRepresentationType

`abstract type AtomicState.AbstractRepresentationType`      ... defines an abstract type and a number of data types to work with and distinguish different atomic representations; see also:

```
+ struct MeanFieldBasis       ... to represent (and generate) a mean-field basis and, especially, a set of 
                                  (mean-field) orbitals.
+ struct OneElectronSpectrum  ... to represent (and generate) a one-electron spectrum for the mean-field 
                                  potential of refConfigs.
+ struct CiExpansion          ... to represent (and generate) a configuration-interaction representation.
+ struct RasExpansion         ... to represent (and generate) a restricted active-space representation.
+ struct GreenExpansion       ... to represent (and generate) an approximate (many-electron) Green functions.
```


Since this is an *abstract type*, we must always specify on of the explicit types together with its parameters as, for instance, a `MeanFieldBasis`, and which is purely determined by its settings:

In [4]:
? MeanFieldBasis

search: MeanFieldBasis



`struct  AtomicState.MeanFieldBasis  <:  AbstractRepresentationType`       ... a struct to represent (and generate) a mean-field orbital basis.

```
+ settings         ::AtomicState.MeanFieldSettings      ... Settings for the given mean-field orbital basis
```


In [5]:
? AtomicState.MeanFieldSettings

`struct  AtomicState.MeanFieldSettings`       ... a struct for defining the settings for a mean-field basis (orbital) representation.

```
+ scField           ::AbstractScField   
    ... Specify the (mean) self-consistent field as DFSField() or HSField(); note that not all AbstractScField's 
        allowed here.
```

---

`AtomicState.MeanFieldSettings()`  ... constructor for setting the default values.


With this background, we can now readily compute a mean-field (orbital) basis for the oxygen ground configuration by:

In [6]:
name        = "Oxygen 1s^2 2s^2 2p^4 ground configuration"
refConfigs  = [Configuration("[He] 2s^2 2p^4")]
mfSettings  = MeanFieldSettings()

scField:                  JAC.Basics.DFSField()  


In [7]:
wa          = Representation(name, Nuclear.Model(8.), Radial.Grid(true), refConfigs, MeanFieldBasis(mfSettings) )

Atomic representation:   Oxygen 1s^2 2s^2 2p^4 ground configuration for Z = 8.0 and with reference configurations: 
   1s^2 2s^2 2p^4 ,  
representation type:   Mean-field orbital basis for a JAC.Basics.DFSField() SCF field:
  
nuclearModel:          Fermi nuclear model for Z = 8.0 with mass = 16.32, radius R = 2.6905393158807427 fm and nuclear spin I = 0, dipole moment mu = 0.0 and quadrupole moment Q = 0.0.  
grid:                  Radial grid:  rnt = 2.0e-6,  h = 0.05,  hp = 0.0,  NoPoints = 392,  ntL = 69,  ntS = 71, orderL = 7,  orderS = 8,  nsL = 62,  nsS = 63,  mesh = JAC.Radial.MeshGL(), ...  
r:    [1.780504510281598e-8, 9.042759120177923e-8, 2.0787030685460475e-7]  ...  [564.4302351890452, 595.097239730825, 614.0606632064549]
wr:   [4.530145586285996e-8, 9.785739473901763e-8, 1.3358660616640612e-7]  ...  [34.882547080539766, 25.552825070730517, 11.829256033233948]
tS:   [0.0, 0.0, 0.0]  ...  [618.7099715607404, 618.7099715607404, 618.7099715607404]  


In JAC, an atomic representation is always *generate*'d:

In [8]:
wb = generate(wa, output=true)


... in performSCF ...
>> include Configuration: 1s_1/2^2 2s_1/2^2 2p_1/2^0 2p_3/2^4 
>> include Configuration: 1s_1/2^2 2s_1/2^2 2p_1/2^1 2p_3/2^3 
>> include Configuration: 1s_1/2^2 2s_1/2^2 2p_1/2^2 2p_3/2^2 
(Re-) Define a new standard subshell list.
Start SCF process with hydrogenic orbitals.
>> (Re-) Define a storage array for various B-spline matrices:
Nuclear model = Fermi nuclear model for Z = 8.0 with mass = 16.32, radius R = 2.6905393158807427 fm and nuclear spin I = 0, dipole moment mu = 0.0 and quadrupole moment Q = 0.0. 
Generate hydrogenic orbital for subshell 1s_1/2 
  -----------------------------------------------------------------------------
   Index    Subshell     Energies [a.u.]    Dirac-E  [a.u.]     Delta-E / |E|    
  -----------------------------------------------------------------------------
      1      1s_1/2      -3.20273042e+01    -3.20273113e+01    +2.19887367e-07    
      2      2s_1/2      -8.00853628e+00    -8.00853549e+00    -9.77892436e-08    
  

     21     22p_3/2      -1.33156548e-02    -6.61204757e-02    +3.96561953e+00    
     22     23p_3/2      +6.82327248e-01    -6.04956840e-02    +1.08866081e+00    
     23     24p_3/2      +6.21303041e+00    -5.55592546e-02    +1.00894238e+00    
     24     25p_3/2      +2.04566662e+01    -5.12032814e-02    +1.00250301e+00    
     25     26p_3/2      +5.30705164e+01    -4.73402024e-02    +1.00089202e+00    
     26     27p_3/2      +1.23880923e+02    -4.38983648e-02    +1.00035436e+00    
     27     28p_3/2      +2.72919476e+02    -4.08186781e-02    +1.00014956e+00    
     28     29p_3/2      +5.79395193e+02    -3.80520613e-02    +1.00006568e+00    
     29     30p_3/2      +1.19525809e+03    -3.55574747e-02    +1.00002975e+00    
     30     31p_3/2      +2.39733320e+03    -3.33003895e-02    +1.00001389e+00    
     31     32p_3/2      +4.65018356e+03    -3.12515865e-02    +1.00000672e+00    
     32     33p_3/2      +8.65086368e+03    -2.93862054e-02    +1.00000340e+00    
    

  1s_1/2::  en [a.u.] = -1.8705398e+01;   self-cons'cy = 5.5904e-07  [2.5939e-03 for sym-block kappa = -1]
  2s_1/2::  en [a.u.] = -8.2174233e-01;   self-cons'cy = 4.8480e-06  [2.5939e-03 for sym-block kappa = -1]
  2p_1/2::  en [a.u.] = -2.8969094e-01;   self-cons'cy = 1.4559e-04  [1.4559e-04 for sym-block kappa = 1]
  2p_3/2::  en [a.u.] = -2.8832483e-01;   self-cons'cy = 1.4338e-04  [1.4338e-04 for sym-block kappa = -2]

Iteration 15 for symmetries ... 
  1s_1/2::  en [a.u.] = -1.8705183e+01;   self-cons'cy = 5.7401e-06  [3.9900e-03 for sym-block kappa = -1]
  2s_1/2::  en [a.u.] = -8.2164980e-01;   self-cons'cy = 5.6301e-05  [3.9900e-03 for sym-block kappa = -1]
  2p_1/2::  en [a.u.] = -2.8961921e-01;   self-cons'cy = 1.2382e-04  [1.2382e-04 for sym-block kappa = 1]
  2p_3/2::  en [a.u.] = -2.8831127e-01;   self-cons'cy = 2.3509e-05  [2.3509e-05 for sym-block kappa = -2]

Iteration 16 for symmetries ... 
  1s_1/2::  en [a.u.] = -1.8705318e+01;   self-cons'cy = 3.6099e-06  [1.8162e-

Dict{String, Any} with 1 entry:
  "mean-field basis" => Atomic basis:  5 CSF defined for 8 elecrons.…

As seen from the last line of this printout, the outcome::Dict{String,Any} just contains the requested basis and, hence, set of orbital:

In [9]:
orbitals    = wb["mean-field basis"].orbitals

Dict{Subshell, Orbital} with 4 entries:
  2s_1/2 => 
  2p_1/2 => 
  2p_3/2 => 
  1s_1/2 => 

We can apply this *mean-field basis* in order to generate a CI representations for all levels of the $1s^2 2s^2 2p^4$ ground configuration of oxygen; to this end, we first define the corresponding *settings* and (no further) *excitations*,

In [10]:
?CiSettings

search: CiSettings



`struct  AtomicState.CiSettings`       ... a struct for defining the settings for a configuration-interaction (CI) expansion.

```
+ eeInteractionCI      ::AbstractEeInteraction   ... Specifies the treatment of the e-e interaction.
+ levelSelectionCI     ::LevelSelection          ... Specifies the selected levels, if any.
```

---

`AtomicState.CiSettings()`  ... constructor for setting the default values.

---

`AtomicState.CiSettings(settings::AtomicState.CiSettings;`

```
    eeInteractionCI::Union{Nothing,AbstractEeInteraction}=nothing,    levelSelectionCI::Union{Nothing,LevelSelection}=nothing)
                
... constructor for modifying the given CiSettings by 'overwriting' the explicitly selected parameters.
```


In [11]:
ciSettings  = CiSettings(CoulombInteraction(), LevelSelection(true, indices=[1,2]) )

eeInteractionCI:          CoulombInteraction()  
levelSelectionCI:         LevelSelection:  indices = [1, 2];    symmetries = LevelSymmetry[].  


In [12]:
excitations = RasStep()


CI or RAS step with 0 (explicitly) frozen shell(s): Shell[]  ... and virtual excitations


and before we follow very similar lines as above:

In [13]:
wc          = Representation(name, Nuclear.Model(8.), Radial.Grid(true), refConfigs, 
                             CiExpansion(orbitals, excitations, ciSettings) )

Atomic representation:   Oxygen 1s^2 2s^2 2p^4 ground configuration for Z = 8.0 and with reference configurations: 
   1s^2 2s^2 2p^4 ,  
representation type:   CI expansion with (additional) excitations:

CI or RAS step with 0 (explicitly) frozen shell(s): Shell[]  ... and virtual excitations
  
... and the current settings:
eeInteractionCI:          CoulombInteraction()  
levelSelectionCI:         LevelSelection:  indices = [1, 2];    symmetries = LevelSymmetry[].  
  
  
nuclearModel:          Fermi nuclear model for Z = 8.0 with mass = 16.32, radius R = 2.6905393158807427 fm and nuclear spin I = 0, dipole moment mu = 0.0 and quadrupole moment Q = 0.0.  
grid:                  Radial grid:  rnt = 2.0e-6,  h = 0.05,  hp = 0.0,  NoPoints = 392,  ntL = 69,  ntS = 71, orderL = 7,  orderS = 8,  nsL = 62,  nsS = 63,  mesh = JAC.Radial.MeshGL(), ...  
r:    [1.780504510281598e-8, 9.042759120177923e-8, 2.0787030685460475e-7]  ...  [564.4302351890452, 595.097239730825, 614.0606632064549]
wr:

In [14]:
wd = generate(wc, output=true)

*** Level symmetries = LevelSymmetry[0 +, 1 +, 2 +] 
>> include Configuration: 1s^2 2s^2 2p^4 
(Re-) Define a new standard subshell list.
Construct a basis with 5 CSF for J^P = LevelSymmetry[0 +, 1 +, 2 +] with 4 subshells: 1s_1/2  2s_1/2 ...  2p_1/2  2p_3/2
Compute CI matrix of dimension 2 x 2 for the symmetry 0^+ ...   ... done.
Compute CI matrix of dimension 1 x 1 for the symmetry 1^+ ...   ... done.
Compute CI matrix of dimension 2 x 2 for the symmetry 2^+ ...   ... done.

  Eigenenergies:

  Level  J Parity          Hartrees                    eV                   [eV]

     1    2 +     -7.484772845491783e+01    -2.036710431706907e+03    -2.036710431706907e+03 
     2    1 +     -7.484696581599319e+01    -2.036689679244735e+03    -2.036689679244735e+03 

  Energy of each level relative to immediately lower level:

  Level  J Parity          Hartrees                    eV                   [eV]

     2    1 +     7.626389246411236e-04    2.075246217228730e-02    2.075246217228730e

Dict{String, Any} with 1 entry:
  "CI multiplet" => name:        CI multiplet:  …

In this case, the outcome::Dict{String,Any} describes the requested (ground configuration) multiplet with the levels:

In [15]:
levels     = wd["CI multiplet"].levels

5-element Vector{Level}:
 Level: J = 2, M = 2, parity = plus, index = 1 
energy:         -74.84772845491783  
relativeOcc:    0.0  
hasStateRep:    true  
basis:           (level.basis)  
mc:             [0.0, 0.0, 0.5715606204119051, 0.0, 0.8205598437617807]  

 Level: J = 1, M = 1, parity = plus, index = 2 
energy:         -74.84696581599319  
relativeOcc:    0.0  
hasStateRep:    true  
basis:           (level.basis)  
mc:             [0.0, 1.0, 0.0, 0.0, 0.0]  

 Level: J = 0, M = 0, parity = plus, index = 3 
energy:         -74.84659206331618  
relativeOcc:    0.0  
hasStateRep:    true  
basis:           (level.basis)  
mc:             [-0.8130771970235597, 0.0, 0.0, 0.5821558826296538, 0.0]  

 Level: J = 2, M = 2, parity = plus, index = 4 
energy:         -74.77019425480822  
relativeOcc:    0.0  
hasStateRep:    true  
basis:           (level.basis)  
mc:             [0.0, 0.0, -0.8205598437617807, 0.0, 0.5715606204119051]  

 Level: J = 0, M = 0, parity = plus, index = 5 
ene

# **This nootebook is still under construction !!!**